In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})
mem = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv',
                  infer_datetime_format=True, parse_dates=[5,61,62],
                 dtype= {6:'str',10:'str',11:'str',13:'str'})
print(admis.shape)
print(visits.shape)
print(mem.shape)
admisans = admis.copy()

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(6213, 23)
(231373, 18)
(61519, 63)


In [4]:
admisd = admis.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
admisd = admisd.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
admisd

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
77,HIA|DS1|10734|DS1|NPI|1104982917|1,DS1|10734,DS1|1,DS1|5047W,DS1|NPI|1104982917,DS1|NPI|1629382395,1629382395,1104982917,2009-11-11,2009-11-25,...,1,01,21,427.31,Atrial fibrillation,NaN,NaN,0,58730.78,0.0
1899,HIA|DS1|26295|DS1|NPI|1689608150|1,DS1|26295,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1346313590,1346313590,1689608150,2009-11-12,2009-11-30,...,1,01,NaN,779.89,Other specified conditions originating in the ...,NaN,NaN,0,32156.07,0.0
4982,HIA|DS1|53976|DS1|NPI|1689772592|1,DS1|53976,DS1|1,DS1|5047W,DS1|NPI|1689772592,DS1|NPI|1144488727,1144488727,1689772592,2009-11-29,2009-12-01,...,1,06,21,285.9,"Anemia, unspecified",NaN,NaN,0,9404.00,250.0
1304,HIA|DS1|21571|DS1|NPI|1689608150|1,DS1|21571,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1376637637,1376637637,1689608150,2009-11-27,2009-12-02,...,4,01,21,493.22,Chronic obstructive asthma with (acute) exacer...,NaN,NaN,0,23949.27,250.0
3429,HIA|DS1|40214|DS1|NPI|1710918545|1,DS1|40214,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1255314399,1255314399,1710918545,2009-12-03,2009-12-05,...,1,01,21,491.21,Obstructive chronic bronchitis with (acute) ex...,NaN,NaN,0,19378.00,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,HIA|DS1|31201|DS1|NPI|1184722779|1,DS1|31201,DS1|1,DS1|5047W,DS1|NPI|1184722779,DS1|NPI|1053334292,1053334292,1184722779,2019-02-15,2019-02-18,...,NaN,01,21,NaN,NaN,Z38.01,"Single liveborn infant, delivered by cesarean",0,2596.78,250.0
5905,HIA|DS1|7017|DS1|NPI|1184722779|1,DS1|7017,DS1|1,DS1|5047X,DS1|NPI|1184722779,DS1|NPI|1730127069,1730127069,1184722779,2019-02-14,2019-02-18,...,1,01,21,NaN,NaN,O43.123,"Velamentous insertion of umbilical cord, third...",0,20933.00,250.0
1335,HIA|DS1|2184|DS1|NPI|1902803315|2,DS1|2184,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1437358884,1437358884,1902803315,2019-02-24,2019-02-26,...,1,06,21,NaN,NaN,Z43.2,Encounter for attention to ileostomy,0,18750.00,250.0
4643,HIA|DS1|51247|DS1|NPI|1184722779|2,DS1|51247,DS1|1,DS1|5047W,DS1|NPI|1184722779,DS1|NPI|1851529879,1851529879,1184722779,2019-04-09,2019-04-13,...,1,01,21,NaN,NaN,T82.855A,"Stenosis of coronary artery stent, initial enc...",0,45083.79,202.0


In [7]:
postvisit = 0

#pull patient numbers that have more then one admission to loop through
for patnum in admis['Patient ID'].drop_duplicates():
    #create dataframe for each patient number keeping only the final discharge per Start Date
    patient = admis[admis['Patient ID']==patnum]
    patient = patient.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
    patient = patient.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
    vpatient = visits[visits['Patient ID']==patnum].sort_values('Encounter Date')
    
    for rnum in range(0,len(patient)):          
        trigger7 = vpatient[vpatient['Encounter Date']>=patient.iloc[rnum,9]]['Encounter Date']  <=  (patient.iloc[rnum,9] + pd.DateOffset(days=7))
        postvisit += int(trigger7.sum() > 0)
        window = (patient.iloc[rnum,9] + pd.DateOffset(days=7))
        if trigger7.sum() > 0:
            admisd.loc[admisd['Encounter ID']==patient.iloc[rnum,0],'pcp_visit'] = True
            
        else:
            admisd.loc[admisd['Encounter ID']==patient.iloc[rnum,0],'pcp_visit'] = False




In [8]:
postvisit

1860

In [9]:
admis[~admis['Encounter ID'].isin(admisd['Encounter ID'])]

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
71,HIA|DS1|10707|DS1|NPI|1992818256|1,DS1|10707,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1194047126,1194047126,1992818256,2010-04-19,2010-05-08,...,5,01,NaN,NaN,NaN,Q25.0,Patent ductus arteriosus,0,0.00,0.0
72,HIA|DS1|10707|DS1|NPI|1992818256|2,DS1|10707,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1932230810,1932230810,1992818256,2010-07-25,2010-07-27,...,2,01,NaN,NaN,NaN,Q87.2,Congenital malformation syndromes predominantl...,0,0.00,0.0
217,HIA|DS1|11785|DS1|NPI|1134564693|2,DS1|11785,DS1|1,DS1|5047X,DS1|NPI|1134564693,DS1|NPI|1558322347,1558322347,1134564693,2014-02-01,2014-03-14,...,4,30,21,518.84,Acute and chronic respiratory failure,NaN,NaN,0,183270.08,0.0
307,HIA|DS1|12521|DS1|NPI|0000000000|1,DS1|12521,DS1|1,DS1|5047Z,DS1|NPI|0000000000,NaN,NaN,0000000000,2013-02-25,2013-03-01,...,1,01,21,641.11,"Hemorrhage from placenta previa, delivered, wi...",NaN,NaN,0,15550.00,250.0
450,HIA|DS1|1364|DS1|NPI|1730238007|1,DS1|1364,DS1|1,DS1|5047Z,DS1|NPI|1730238007,DS1|NPI|1891722211,1891722211,1730238007,2011-06-11,2011-06-11,...,1,02,NaN,NaN,NaN,O42.912,"Preterm premature rupture of membranes, unspec...",0,4358.08,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5508,HIA|DS1|58868|DS1|NPI|1629011945|1,DS1|58868,DS1|1,DS1|5047X,DS1|NPI|1629011945,DS1|NPI|1750486189,1750486189,1629011945,2011-04-22,2011-04-25,...,1,01,NaN,NaN,NaN,D64.9,"Anemia, unspecified",0,0.00,0.0
5560,HIA|DS1|59409|DS1|NPI|0000000000|1,DS1|59409,DS1|1,DS1|5047W,DS1|NPI|0000000000,NaN,NaN,0000000000,2011-11-23,2011-11-25,...,NaN,01,21,NaN,NaN,Z38.00,"Single liveborn infant, delivered vaginally",0,920.81,0.0
5612,HIA|DS1|59805|DS1|NPI|1689772592|1,DS1|59805,DS1|1,DS1|5047X,DS1|NPI|1689772592,NaN,NaN,1689772592,2014-09-28,2014-10-18,...,2,01,NaN,NaN,NaN,C84.41,"Peripheral T-cell lymphoma, not classified, ly...",0,231000.50,250.0
5898,HIA|DS1|6964|DS1|NPI|0000000000|1,DS1|6964,DS1|1,DS1|5047X,DS1|NPI|0000000000,NaN,NaN,0000000000,2018-07-21,2018-09-18,...,NaN,30,21,NaN,NaN,Z38.00,"Single liveborn infant, delivered vaginally",0,23621.66,0.0


In [10]:
admisd['pcp_visit'].sum()

1860

In [11]:
vpatient[vpatient['Encounter Date']>=patient.iloc[rnum,9]]['Encounter Date'] 

Series([], Name: Encounter Date, dtype: datetime64[ns])